In [7]:
import yfinance as yf
import sqlite3
import os
from pathlib import Path
import datetime
import pandas as pd
import numpy as np
import datetime,time,re
from sklearn.linear_model import LinearRegression
import requests
from bs4 import BeautifulSoup
today = datetime.date.today()
end = today - datetime.timedelta(days=1)
start = end - datetime.timedelta(days=365*3.5)

In [2]:
def five_line(data):   
    timetrend = list(range(1, data.shape[0]+1))
    data['timetrend'] = timetrend
    data = data[['timetrend','Close']]
    data = data.dropna()
    reg = LinearRegression()
    x = data['timetrend'].to_frame()
    y = data['Close'].to_frame()
    reg.fit(x,y)
    
    a = reg.intercept_ #截距
    beta = reg.coef_ #斜率
    #print(beta)
    #beta = beta[0][0]
    #print(beta)
    longtrend = a + beta*x
    res = np.array(list(data['Close'])) - np.array(list(longtrend['timetrend']))
    std = np.std(res,ddof=1)
    fiveline = pd.DataFrame()
    fiveline['highest'] = longtrend['timetrend'] + (2*std)
    fiveline['high'] = longtrend['timetrend'] + (1*std)
    fiveline['TL'] = longtrend['timetrend']
    fiveline['low'] = longtrend['timetrend'] - (1*std)
    fiveline['lowest'] = longtrend['timetrend'] - (2*std) 
    use_fiveline = pd.merge(data, fiveline[['highest','high','TL','low','lowest']], left_index=True, right_index=True, how='left')
    pick_fiveline = use_fiveline[['Close','highest','high','TL','low','lowest']]
    return pick_fiveline,beta

def stock_price(stock:str):
    headers={
        "User-Agent":"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.146 Safari/537.36"
    }

    data = requests.get(f"https://finance.yahoo.com/quote/{stock}.TW?p={stock}.TW", headers = headers)
    soup = BeautifulSoup(data.text)
    price = soup.find('fin-streamer',{'class':'Fw(b) Fz(36px) Mb(-4px) D(ib)'})
    return price.text

In [14]:
df = yf.download(f"0050.TW", start = str(start), end = str(end))
df2,beta = five_line(df)
print(df2)

[*********************100%***********************]  1 of 1 completed
                 Close     highest        high          TL         low  \
Date                                                                     
2018-12-19   75.900002   89.322471   79.363627   69.404783   59.445940   
2018-12-20   75.050003   89.416680   79.457836   69.498992   59.540148   
2018-12-21   75.199997   89.510888   79.552045   69.593201   59.634357   
2018-12-24   74.699997   89.605097   79.646253   69.687410   59.728566   
2018-12-25   73.750000   89.699306   79.740462   69.781619   59.822775   
...                ...         ...         ...         ...         ...   
2022-06-13  123.550003  168.457844  158.499000  148.540157  138.581313   
2022-06-14  123.349998  168.552053  158.593209  148.634365  138.675522   
2022-06-15  122.750000  168.646262  158.687418  148.728574  138.769731   
2022-06-16  122.500000  168.740470  158.781627  148.822783  138.863939   
2022-06-17  120.849998  168.834679  158.875

In [8]:
conn = sqlite3.connect('/home/awinlab/Documents/yeh/LineBot/FiveLineAPP/ETF_list.db')
cursor = conn.cursor()

In [19]:
sql =f"select * from Strategy_data where Number = '0050'"
cursor.execute(sql)
data = cursor.fetchall()
print(data)

[('2019-01-16', '0050', 75.55000305175781, 90.89360426463477, 80.97977412998868, 71.0659439953426, 61.1521138606965, 51.238283726050405, 76.5159859157208, 74.63500061035157, 72.75401530498233, 84.03063028826773, 72.06179965553642), ('2019-01-17', '0050', 75.55000305175781, 90.98799890057043, 81.07416876592434, 71.16033863127825, 61.24650849663216, 51.33267836198606, 76.54920536218756, 74.64750061035156, 72.74579585851555, 84.95125953065758, 76.3582862805768), ('2019-01-18', '0050', 75.94999694824219, 91.08239353650609, 81.16856340186, 71.25473326721391, 61.34090313256782, 51.42707299792172, 76.55840379822871, 74.65000038146972, 72.74159696471074, 88.0066985739694, 80.24109037837434), ('2019-01-21', '0050', 76.5, 91.17678817244175, 81.26295803779566, 71.34912790314957, 61.435297768503474, 51.52146763385738, 76.7897394458239, 74.72250022888184, 72.65526101193979, 89.58026028274097, 83.35414701316321), ('2019-01-22', '0050', 74.05000305175781, 91.27118280837743, 81.35735267373133, 71.4435

In [31]:
df2 = pd.DataFrame(data)
df3 = df2.drop([1,8,9,10,11,12],axis =1)
#df3 = df2.iloc[:][0,2,3,4,5,6,7]
df3.set_axis(['Date', 'Close', 'highest','high','TL','low','lowest'], 
                    axis='columns', inplace=True)
df3.set_index('Date', inplace = True)
print(df3.empty)

False
